# Limpando os dados

In [1]:
import pandas as pd

from pathlib import Path
from datetime import datetime, timedelta

### Lendo arquivos em dataframes

In [2]:
!ls data

01-12-2020_a_31-01-2021_comparacao_estados.csv
01-12-2020_a_31-01-2021_comparacao_regioes.csv
clean_data.csv
HIST_PAINEL_COVIDBR_2020_Parte1_10jul2021.csv
HIST_PAINEL_COVIDBR_2020_Parte2_10jul2021.csv
HIST_PAINEL_COVIDBR_Parte3_10jul2021.csv


In [3]:
UFS = ['AC','AL','AP','AM','BA','CE','DF','ES','GO','MA','MT','MS','MG','PA','PB','PR','PE','PI','RJ','RN','RS','RO','RR','SC','SP','SE','TO']
REGIOES= ['Brasil','Norte', 'Nordeste', 'Centro-Oeste', 'Sul', 'Sudeste']

# Makes it compatible with both windows and linux environments
data_path = Path("data")

parte_1 = pd.read_csv(data_path / "HIST_PAINEL_COVIDBR_2020_Parte1_10jul2021.csv", sep=";") #dados dos estados acabam na linha 4423
parte_1 = parte_1.loc[0:4423]

parte_2 = pd.read_csv(data_path / "HIST_PAINEL_COVIDBR_2020_Parte2_10jul2021.csv", sep=";") #dados dos estados acabam na linha 4284
parte_2 = parte_2.loc[0:4284]

parte_3 = pd.read_csv(data_path / "HIST_PAINEL_COVIDBR_2021_Parte1_15ago2021.csv", sep=";") #dados dos estados acabam na linha 5067
parte_3 = parte_3.loc[0:5067]

### Removendo colunas desnecessárias do dado

In [4]:
df_total = pd.concat([parte_1, parte_2, parte_3], ignore_index=True)
df_total.data = df_total.data.apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
df_total.drop(columns=["municipio", "coduf", "codmun", "codRegiaoSaude","nomeRegiaoSaude", "Recuperadosnovos", "emAcompanhamentoNovos", "interior/metropolitana"], inplace=True)
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14058 entries, 0 to 14057
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   regiao            14058 non-null  object        
 1   estado            13554 non-null  object        
 2   data              14058 non-null  datetime64[ns]
 3   semanaEpi         14058 non-null  int64         
 4   populacaoTCU2019  14058 non-null  float64       
 5   casosAcumulado    14058 non-null  float64       
 6   casosNovos        14058 non-null  int64         
 7   obitosAcumulado   14058 non-null  int64         
 8   obitosNovos       14058 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(4), object(2)
memory usage: 988.6+ KB


In [6]:
df_total.head()

,regiao,estado,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos
0,Brasil,NaN,2020-02-25,9,210147125.0,0.0,0,0,0
1,Brasil,NaN,2020-02-26,9,210147125.0,1.0,1,0,0
2,Brasil,NaN,2020-02-27,9,210147125.0,1.0,0,0,0
3,Brasil,NaN,2020-02-28,9,210147125.0,1.0,0,0,0
4,Brasil,NaN,2020-02-29,9,210147125.0,2.0,1,0,0


In [11]:
df_total.to_csv("data/clean_data.csv",sep=";",index=False)

# Gerando tabelas para os gráficos

In [12]:
def pega_linha_com_indices(df_uf):
    populacao = df_uf["populacaoTCU2019"].iloc[0]
    casos_periodo = df_uf["casosNovos"].sum()
    obitos_periodo = df_uf["obitosNovos"].sum()
    obitos_por_casos = obitos_periodo/casos_periodo*100
    casos_100_mil = casos_periodo/populacao*100000
    obitos_100_mil = obitos_periodo/populacao*100000
    linha = {"população": populacao,
            "casosPeríodo": casos_periodo,
            "óbitosPeríodo": obitos_periodo,
            "óbitosPorCasos": obitos_por_casos,
            "casosPorCemMilHab": casos_100_mil,
            "óbitosPorCemMilHab": obitos_100_mil}
    return linha

In [13]:
def gera_resultados_uf(data_inicial, data_final, df_total):
    df_parcial = df_total.loc[(df_total['data']>= data_inicial) & (df_total['data']<= data_final)]
    lista_resultados = []
    dict_br = {"UF": 'Brasil', 'região': "Brasil"}
    df_br = df_parcial.loc[(pd.isna(df_parcial["estado"]))]
    dict_br.update(pega_linha_com_indices(df_br))
    lista_resultados.append(dict_br)
    for uf in UFS:
        df_uf = df_parcial.loc[(df_parcial["estado"]==uf)]
        dict_uf = {"UF": uf, "região": df_uf["regiao"].iloc[0]}
        dict_uf.update(pega_linha_com_indices(df_uf))
        lista_resultados.append(dict_uf)
    df_resultados = pd.DataFrame.from_records(lista_resultados)
    return df_resultados

In [14]:
def gera_resultados_regiao(df_uf):
    lista_resultados = []
    for regiao in REGIOES:
        df_reg = df_uf.loc[(df_uf["região"]==regiao)]
        dict_reg = {"região": regiao}
        dict_reg['população'] = df_reg['população'].sum()
        dict_reg['casosPeríodo'] = df_reg["casosPeríodo"].sum()
        dict_reg['óbitosPeríodo'] = df_reg["óbitosPeríodo"].sum()
        dict_reg['óbitosPorCasos'] = dict_reg['óbitosPeríodo']/dict_reg['casosPeríodo']*100
        dict_reg['casosPorCemMilHab'] = dict_reg['casosPeríodo']/dict_reg['população']*100000
        dict_reg['óbitosPorCemMilHab'] = dict_reg['óbitosPeríodo']/dict_reg['população']*100000
        lista_resultados.append(dict_reg)
    df_resultados = pd.DataFrame.from_records(lista_resultados)
    return df_resultados

In [15]:
data_inicial = datetime(2020,12,1)
data_final = datetime(2021,1,31)
df_resultados_uf = gera_resultados_uf(data_inicial, data_final , df_total)
df_resultados_uf

,UF,região,população,casosPeríodo,óbitosPeríodo,óbitosPorCasos,casosPorCemMilHab,óbitosPorCemMilHab
0,Brasil,Brasil,210147125.0,2868853,51384,1.791099,1365.164049,24.451441
1,AC,Norte,881935.0,12208,144,1.179554,1384.228996,16.327734
2,AL,Nordeste,3337357.0,22784,407,1.786341,682.695918,12.195279
3,AP,Norte,845731.0,17910,252,1.407035,2117.694633,29.796708
4,AM,Norte,4144597.0,89194,3223,3.613472,2152.054832,77.763894
5,BA,Nordeste,14873064.0,185035,1829,0.988462,1244.094694,12.297399
6,CE,Nordeste,9132078.0,72790,864,1.186976,797.080358,9.461154
7,DF,Centro-Oeste,3015268.0,47964,624,1.300976,1590.704375,20.694678
8,ES,Sudeste,4018650.0,104215,1585,1.520894,2593.283814,39.441106
9,GO,Centro-Oeste,7018354.0,71156,1137,1.597898,1013.855955,16.200380


In [16]:
df_resultados_regioes = gera_resultados_regiao(df_resultados_uf)
df_resultados_regioes

,região,população,casosPeríodo,óbitosPeríodo,óbitosPorCasos,casosPorCemMilHab,óbitosPorCemMilHab
0,Brasil,210147125.0,2868853,51384,1.791099,1365.164049,24.451441
1,Norte,18430980.0,253573,5370,2.117733,1375.797706,29.135727
2,Nordeste,57071654.0,544890,7074,1.298244,954.747167,12.394945
3,Centro-Oeste,16297074.0,238752,3866,1.619253,1464.999177,23.722050
4,Sul,29975984.0,706944,10310,1.458390,2358.367952,34.394200
5,Sudeste,88371433.0,1124694,24764,2.201843,1272.689558,28.022630


In [9]:
nome_arquivo = data_inicial.strftime("%d-%m-%Y")+'_a_'+data_final.strftime("%d-%m-%Y")+'_comparacao_estados.csv'
df_resultados_uf.to_csv("data/"+nome_arquivo, index=False)

In [10]:
nome_arquivo = data_inicial.strftime("%d-%m-%Y")+'_a_'+data_final.strftime("%d-%m-%Y")+'_comparacao_regioes.csv'
df_resultados_regioes.to_csv("data/"+nome_arquivo, index=False)